In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Function to remove symbols from the word
def remove_symbols(word):
    return ''.join(char for char in word if (
        ('\u1200' <= char <= '\u137C') and
        (char not in ('\u1362', '\u1363', '\u1366', '\u1367', '\u2018', '\u2019', '\u1364'))
    ) or char.isspace())

# Function to clean the text
def clean_text(line):
    words = line.split() if len(line) > 1 else []
    cleaned_words = [word for word in words if remove_symbols(word)]
    return ' '.join(cleaned_words)

# Load the labeled dataset for text classification (assuming the format is "text \t label")
# Replace 'your_dataset.txt' with the actual path and filename of your labeled dataset
dataset_path = 'your_dataset.txt'
with open(dataset_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Split the lines into text and label
data = [line.strip().split('\t') for line in lines if len(line.strip().split('\t')) == 2]

# Clean the text in the dataset
data = [(clean_text(text), label) for text, label in data]

# Print the number of samples before the split
print("Number of samples before split:", len(data))

# Check if the dataset size is sufficient for the split
if len(data) == 0:
    raise ValueError("The dataset is empty. Please check your data cleaning process.")
elif len(data) < 5:
    raise ValueError("The dataset is too small. Ensure you have enough labeled samples for training and validation.")

# Split the dataset into training and validation sets
train_data, valid_data = train_test_split(data, test_size=0.2, random_state=42)

# Extract text and labels for training
X_train, y_train = zip(*train_data)

# Extract text and labels for validation
X_valid, y_valid = zip(*valid_data)

# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_valid = vectorizer.transform(X_valid)

# Train a simple text classifier (Naive Bayes in this example)
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Evaluate the classifier on the validation set
predictions = classifier.predict(X_valid)
accuracy = accuracy_score(y_valid, predictions)

print(f'Accuracy on Validation Set: {accuracy:.2f}')

# Additional evaluation metrics
print('\nClassification Report:')
print(classification_report(y_valid, predictions))
